# सत्र ४ – SLM बनाम LLM तुलना

एक सानो भाषा मोडेल र Foundry Local मार्फत चलिरहेको ठूलो मोडेलको बीचमा विलम्बता र नमूना प्रतिक्रिया गुणस्तर तुलना गर्नुहोस्।


## ⚡ छिटो सुरु गर्नुहोस्

**स्मृति-अनुकूल सेटअप (अपडेट गरिएको):**
1. मोडेलहरूले CPU भेरियन्टहरू स्वचालित रूपमा चयन गर्छन् (कुनै पनि हार्डवेयरमा काम गर्छ)
2. `qwen2.5-3b` प्रयोग गर्दछ 7B को सट्टा (झन्डै 4GB RAM बचत गर्छ)
3. पोर्ट स्वचालित रूपमा पत्ता लगाउने सुविधा (म्यानुअल कन्फिगरेसन आवश्यक छैन)
4. कुल RAM आवश्यक: ~8GB सिफारिस गरिएको (मोडेलहरू + OS)

**टर्मिनल सेटअप (३० सेकेन्ड):**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

त्यसपछि यो नोटबुक चलाउनुहोस्! 🚀


### व्याख्या: निर्भरता स्थापना
समय र च्याट अनुरोधहरूको लागि आवश्यक न्यूनतम प्याकेजहरू (`foundry-local-sdk`, `openai`, `numpy`) स्थापना गर्दछ। यो पुन: चलाउन सुरक्षित छ।


# परिदृश्य
एउटा प्रतिनिधि सानो भाषा मोडेल (SLM) र ठूलो मोडेललाई एउटै प्रम्प्टमा तुलना गरेर व्यापार‑अफहरू देखाउनुहोस्:
- **ढिलाइको भिन्नता** (वाल क्लक सेकेन्डमा)
- **टोकन प्रयोग** (यदि उपलब्ध छ) throughput को संकेतको रूपमा
- **नमूना गुणात्मक आउटपुट** छिटो मूल्याङ्कनका लागि
- **गतिको गणना** प्रदर्शन लाभलाई मापन गर्न

**पर्यावरण चरहरू:**
- `SLM_ALIAS` - सानो भाषा मोडेल (पूर्वनिर्धारित: phi-4-mini, ~4GB RAM)
- `LLM_ALIAS` - ठूलो भाषा मोडेल (पूर्वनिर्धारित: qwen2.5-7b, ~7GB RAM)
- `COMPARE_PROMPT` - तुलना परीक्षणका लागि प्रम्प्ट
- `COMPARE_RETRIES` - लचिलोताका लागि पुन: प्रयास (पूर्वनिर्धारित: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - सेवा अन्तर्क्रियालाई ओभरराइड गर्नुहोस् (सेट नगरिएमा स्वचालित रूपमा पत्ता लगाइन्छ)

**यसले कसरी काम गर्छ (आधिकारिक SDK ढाँचा):**
1. **FoundryLocalManager** ले Foundry Local सेवाको सुरुवात र व्यवस्थापन गर्छ
2. सेवा स्वचालित रूपमा सुरु हुन्छ यदि चलिरहेको छैन भने (म्यानुअल सेटअप आवश्यक छैन)
3. मोडेलहरू उपनामबाट ठोस ID हरूमा स्वचालित रूपमा समाधान गरिन्छ
4. हार्डवेयर-अनुकूलित भेरियन्टहरू चयन गरिन्छ (CUDA, NPU, वा CPU)
5. OpenAI-संगत ग्राहकले च्याट पूरा गर्दछ
6. मेट्रिक्सहरू क्याप्चर गरिन्छ: ढिलाइ, टोकन, आउटपुट गुणस्तर
7. नतिजाहरू तुलना गरिन्छ गतिको अनुपात गणना गर्न

यो सानो तुलना तपाईंको प्रयोगको केसका लागि ठूलो मोडेलमा रुटिङ कहिले उचित हुन्छ भनेर निर्णय गर्न मद्दत गर्छ।

**SDK सन्दर्भ:**
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- कार्यशाला उपयोगिताहरू: ../samples/workshop_utils.py बाट आधिकारिक ढाँचा प्रयोग गर्दछ

**मुख्य फाइदाहरू:**
- ✅ स्वचालित सेवा पत्ता लगाउने र सुरुवात गर्ने
- ✅ सेवा चलिरहेको छैन भने स्वचालित रूपमा सुरु गर्ने
- ✅ मोडेल समाधान र क्यासिङमा निर्मित
- ✅ हार्डवेयर अनुकूलन (CUDA/NPU/CPU)
- ✅ OpenAI SDK संगतता
- ✅ पुन: प्रयाससहितको बलियो त्रुटि व्यवस्थापन
- ✅ स्थानीय अनुमान (क्लाउड API आवश्यक छैन)


## 🚨 पूर्वशर्त: Foundry Local चलिरहेको हुनुपर्छ!

**यो नोटबुक चलाउनु अघि**, Foundry Local सेवा सेटअप गरिएको सुनिश्चित गर्नुहोस्:

### छिटो सुरु गर्ने कमाण्डहरू (टर्मिनलमा चलाउनुहोस्):

```bash
# 1. Start the Foundry Local service
foundry service start

# 2. Load the default models used in this comparison (CPU-optimized)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. Verify models are loaded
foundry model ls

# 4. Check service health
foundry service status
```

### वैकल्पिक मोडेलहरू (यदि डिफल्ट उपलब्ध छैनन् भने):

```bash
# Even smaller alternatives (if memory is very limited)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# Or update the environment variables in this notebook:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # Or qwen2.5-0.5b for minimum memory
```

⚠️ **यदि तपाईंले यी चरणहरू छोड्नुभयो भने**, तलका नोटबुक सेलहरू चलाउँदा `APIConnectionError` देखिनेछ।


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### व्याख्या: कोर आयातहरू
समय उपयोगिताहरू र Foundry Local / OpenAI क्लाइन्टहरू ल्याउँछ, जसले मोडेल जानकारी प्राप्त गर्न र च्याट पूरा गर्न प्रयोग गरिन्छ।


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### व्याख्या: उपनामहरू र प्रॉम्प्ट सेटअप  
सानो र ठूलो मोडेलका लागि वातावरण-अनुकूलन योग्य उपनामहरू र तुलना प्रॉम्प्ट परिभाषित गर्दछ। विभिन्न मोडेल परिवारहरू वा कार्यहरूमा प्रयोग गर्न वातावरणीय भेरिएबलहरू समायोजन गर्नुहोस्।


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 मेमोरी-अप्टिमाइज्ड कन्फिगरेसन

**यो नोटबुकले डिफल्ट रूपमा मेमोरी-कुशल मोडेलहरू प्रयोग गर्दछ:**
- `phi-4-mini` → ~4GB RAM (Foundry Local CPU भेरियन्टलाई स्वतः चयन गर्दछ)
- `qwen2.5-3b` → ~3GB RAM (7B को सट्टा, जसलाई ~7GB+ आवश्यक पर्छ)

**पोर्ट स्वतः-पहिचान:**
- Foundry Local ले विभिन्न पोर्टहरू प्रयोग गर्न सक्छ (सामान्यतया 55769 वा 59959)
- तलको डायग्नोस्टिक सेलले सही पोर्ट स्वतः पहिचान गर्दछ
- कुनै पनि म्यानुअल कन्फिगरेसन आवश्यक छैन!

**यदि तपाईंको RAM सीमित छ (<8GB), अझ साना मोडेलहरू प्रयोग गर्नुहोस्:**
```python
SLM = 'phi-3.5-mini'      # ~2GB
LLM = 'qwen2.5-0.5b'      # ~500MB
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### व्याख्या: कार्यान्वयन सहायक (Foundry SDK ढाँचा)
Workshop नमूनाहरूमा दस्तावेज गरिएको आधिकारिक Foundry Local SDK ढाँचाको प्रयोग:

**पद्धति:**
- **FoundryLocalManager** - Foundry Local सेवा आरम्भ र व्यवस्थापन गर्दछ
- **स्वत: पहिचान** - स्वत: अन्तर्क्रियालाई पत्ता लगाउँछ र सेवा जीवनचक्रलाई व्यवस्थापन गर्दछ
- **मोडेल समाधान** - उपनामलाई पूर्ण मोडेल ID मा रूपान्तरण गर्दछ (जस्तै, phi-4-mini → phi-4-mini-instruct-cpu)
- **हार्डवेयर अनुकूलन** - उपलब्ध हार्डवेयरका लागि उत्तम भेरियन्ट चयन गर्दछ (CUDA, NPU, वा CPU)
- **OpenAI क्लाइन्ट** - OpenAI-संगत API पहुँचको लागि प्रबन्धकको अन्तर्क्रियासँग कन्फिगर गरिएको

**लचिलोपन सुविधाहरू:**
- एक्स्पोनेंशियल ब्याकअफ पुन: प्रयास तर्क (पर्यावरण मार्फत कन्फिगर गर्न सकिने)
- सेवा चलिरहेको छैन भने स्वत: सेवा सुरु
- आरम्भ पछि जडानको सत्यापन
- विस्तृत त्रुटि रिपोर्टिङसहितको सहज त्रुटि व्यवस्थापन
- बारम्बार आरम्भलाई रोक्न मोडेल क्यासिङ

**परिणाम संरचना:**
- विलम्ब मापन (वाल घडी समय)
- टोकन प्रयोग ट्र्याकिङ (यदि उपलब्ध छ भने)
- नमूना आउटपुट (पढ्न सजिलो बनाउन छोट्याइएको)
- असफल अनुरोधहरूको लागि त्रुटि विवरण

यो ढाँचाले workshop_utils मोड्युललाई उपयोग गर्दछ, जसले आधिकारिक SDK ढाँचाको पालना गर्दछ।

**SDK सन्दर्भ:**
- मुख्य रिपोजिटरी: https://github.com/microsoft/Foundry-Local
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### व्याख्या: प्रि-फ्लाइट सेल्फ-टेस्ट
FoundryLocalManager प्रयोग गरेर दुवै मोडेलहरूको लागि हल्का कनेक्टिभिटी जाँच चलाउँछ। यसले निम्न कुराहरू प्रमाणित गर्छ:
- सेवा पहुँचयोग्य छ
- मोडेलहरू सुरु गर्न सकिन्छ
- उपनामहरू वास्तविक मोडेल आईडीहरूमा समाधान हुन्छन्
- तुलना चलाउनु अघि जडान स्थिर छ

setup() फङ्सनले workshop_utils बाट आधिकारिक SDK ढाँचा प्रयोग गर्छ।


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ छिटो समाधान: नोटबुकबाट स्थानीय फाउन्ड्री सुरु गर्नुहोस् (वैकल्पिक)

यदि माथिको डायग्नोस्टिकले सेवा चलिरहेको छैन भनेर देखाउँछ भने, तपाईं यसलाई यहाँबाट सुरु गर्न प्रयास गर्न सक्नुहुन्छ:

**नोट:** यो विन्डोजमा सबैभन्दा राम्रो काम गर्छ। अन्य प्लेटफर्महरूमा, टर्मिनल कमाण्डहरू प्रयोग गर्नुहोस्।


### ⚠️ जडान त्रुटिहरू समाधान गर्ने

यदि तपाईंले `APIConnectionError` देख्नुभएको छ भने, Foundry Local सेवा चलिरहेको छैन वा मोडेलहरू लोड गरिएको छैन। यी चरणहरू प्रयास गर्नुहोस्:

**1. सेवा स्थिति जाँच गर्नुहोस्:**
```bash
# In a terminal (not in notebook):
foundry service status
```

**2. सेवा सुरु गर्नुहोस् (यदि चलिरहेको छैन भने):**
```bash
foundry service start
```

**3. आवश्यक मोडेलहरू लोड गर्नुहोस्:**
```bash
# Load the models needed for comparison
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# Or use alternative models:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. मोडेलहरू उपलब्ध छन् कि छैनन् पुष्टि गर्नुहोस्:**
```bash
foundry model ls
```

**सामान्य समस्याहरू:**
- ❌ सेवा चलिरहेको छैन → `foundry service start` चलाउनुहोस्
- ❌ मोडेलहरू लोड गरिएको छैन → `foundry model run <model-name>` चलाउनुहोस्
- ❌ पोर्टको द्वन्द्व → अर्को सेवा पोर्ट प्रयोग गरिरहेको छ कि छैन जाँच गर्नुहोस्
- ❌ फायरवालले रोकिरहेको छ → स्थानीय जडानहरू अनुमति दिइएको छ कि छैन सुनिश्चित गर्नुहोस्

**छिटो समाधान:** प्रि-फ्लाइट जाँच अघि तलको डायग्नोस्टिक सेल चलाउनुहोस्।


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ प्रि-फ्लाइट जाँच: मोडेल उपलब्धता

यो सेलले तुलना सुरु गर्नु अघि दुवै मोडेलहरू कन्फिगर गरिएको अन्त बिन्दुमा पहुँचयोग्य छन् कि छैनन् भनेर पुष्टि गर्दछ।


### व्याख्या: रन तुलना र परिणाम संकलन
दुवै उपनामहरूमा आधिकारिक Foundry SDK ढाँचा प्रयोग गर्दै पुनरावृत्ति गर्दछ:
1. प्रत्येक मोडेललाई setup() प्रयोग गरेर आरम्भ गर्नुहोस् (FoundryLocalManager प्रयोग गर्दछ)
2. OpenAI-संग मिल्ने API प्रयोग गरेर अनुमान चलाउनुहोस्
3. विलम्बता, टोकनहरू, र नमूना आउटपुट कैद गर्नुहोस्
4. तुलनात्मक विश्लेषणसहित JSON सारांश उत्पादन गर्नुहोस्

यो session04/model_compare.py मा कार्यशालाका नमूनाहरूको समान ढाँचालाई पछ्याउँछ।


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### नतिजा व्याख्या गर्दै

**मुख्य मापदण्डहरू:**
- **लेटेन्सी**: कम हुनु राम्रो - छिटो प्रतिक्रिया समय देखाउँछ
- **टोकनहरू**: उच्च थ्रुपुट = बढी टोकनहरू प्रशोधन गरिन्छ
- **रूट**: कुन API अन्त बिन्दु प्रयोग गरिएको थियो भनेर पुष्टि गर्छ

**SLM बनाम LLM कहिले प्रयोग गर्ने:**
- **SLM (साना भाषा मोडेल)**: छिटो प्रतिक्रिया, कम स्रोतको प्रयोग, साधारण कार्यहरूको लागि उपयुक्त
- **LLM (ठूला भाषा मोडेल)**: उच्च गुणस्तर, राम्रो तर्क क्षमता, जब गुणस्तर सबैभन्दा महत्त्वपूर्ण हुन्छ तब प्रयोग गर्नुहोस्

**अगाडि के गर्ने:**
1. विभिन्न प्रॉम्प्टहरू प्रयास गर्नुहोस् र जटिलताले तुलना कसरी प्रभावित गर्छ हेर्नुहोस्
2. अन्य मोडेल जोडीहरूमा प्रयोग गर्नुहोस्
3. कार्यको जटिलताका आधारमा बुद्धिमानीपूर्वक रूट गर्न Workshop राउटर नमूनाहरू (Session 06) प्रयोग गर्नुहोस्


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धता हुन सक्छ। मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
